In [3]:
%pip install dash
%pip install dash==1.19.0
%pip install jupyter_dash
%pip install --upgrade plotly


  Using cached dash-1.19.0-py3-none-any.whl
  Using cached dash_core_components-1.15.0-py3-none-any.whl
  Using cached dash_html_components-1.1.2-py3-none-any.whl
  Using cached dash_table-4.11.2-py3-none-any.whl
  Attempting uninstall: dash-table
    Found existing installation: dash-table 5.0.0
    Uninstalling dash-table-5.0.0:
      Successfully uninstalled dash-table-5.0.0
  Attempting uninstall: dash-html-components
    Found existing installation: dash-html-components 2.0.0
    Uninstalling dash-html-components-2.0.0:
      Successfully uninstalled dash-html-components-2.0.0
  Attempting uninstall: dash-core-components
    Found existing installation: dash-core-components 2.0.0
    Uninstalling dash-core-components-2.0.0:
      Successfully uninstalled dash-core-components-2.0.0
  Attempting uninstall: dash
    Found existing installation: dash 2.14.1
    Uninstalling dash-2.14.1:
      Successfully uninstalled dash-2.14.1
Note: you may need to restart the kernel to use updated

In [5]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [1]:
# Import required libraries
import pandas as pd
import dash 
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

In [2]:
!pip list

Package               Version
--------------------- ---------
ansi2html             1.8.0
asttokens             2.2.1
backcall              0.2.0
beautifulsoup4        4.12.2
blinker               1.6.2
branca                0.6.0
Brotli                1.1.0
caer                  2.0.8
certifi               2023.5.7
charset-normalizer    3.1.0
click                 8.1.3
colorama              0.4.6
comm                  0.1.2
contourpy             1.0.7
cycler                0.11.0
dash                  1.19.0
dash-core-components  1.15.0
dash-html-components  1.1.2
dash-renderer         1.9.0
dash-table            4.11.2
debugpy               1.6.6
decorator             5.1.1
executing             1.2.0
fanstatic             1.4
Flask                 2.0.3
Flask-Compress        1.14
folium                0.14.0
fonttools             4.38.0
future                0.18.3
greenlet              3.0.1
ibm-db                3.2.0
ibm-db-sa             0.4.0
idna                  3.4
importli

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)



# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[
                                    {'label': 'All Sites', 'value': 'All Sites'},
                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                ],
                                placeholder='Select a Launch Site Here',
                                value='All Sites',
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(launch_site):
    if launch_site == 'All Sites':
        fig = px.pie(values=spacex_df.groupby('Launch Site')['class'].mean(), 
                     names=spacex_df.groupby('Launch Site')['Launch Site'].first(),
                     title='Total Success Launches by Site')
    else:
        fig = px.pie(values=spacex_df[spacex_df['Launch Site']==str(launch_site)]['class'].value_counts(normalize=True), 
                     names=spacex_df['class'].unique(), 
                     title='Total Success Launches for Site {}'.format(launch_site))
    return(fig)

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider',component_property='value')])
def get_payload_chart(launch_site, payload_mass):
    if launch_site == 'All Sites':
        fig = px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])], 
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for All Sites')
    else:
        df = spacex_df[spacex_df['Launch Site']==str(launch_site)]
        fig = px.scatter(df[df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])], 
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for Site {}'.format(launch_site))
    return(fig)


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2023 13:22:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 13:22:58] "POST /_dash-update-component HTTP/1.1" 200 -
